In [4]:
import modelinference
import prompts
import importlib
import datetime
import torch
from huggingface_hub import login
import utils.modelHelper as mh


from accelerate import Accelerator, notebook_launcher

In [12]:
importlib.reload(modelinference)

<module 'modelinference' from '/project/modelinference.py'>

In [6]:
# Log into Huggingface

with open('pass.txt') as p:
    hf_login = p.read()
    
hf_login = hf_login[hf_login.find('=')+1:hf_login.find('\n')]
login(hf_login, add_to_git_credential=False)

In [7]:
torch.cuda.device_count()

4

In [8]:

run_config = {
    'model_hf_id': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-14B',
    'model_s3_loc': 'deepseek14Q',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['CoT'],
    'multi-gpu':True,
    'dataset': 'data_annual_pit_spx',
    'data_location': 'data_annual_pit_spx.json'
}

run_config = {
    'model_hf_id': 'Qwen/Qwen2.5-7B-Instruct',
    'model_s3_loc': 'qwen',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['BASE'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu',
    'data_location': 'data_quarterly_pit_indu.json'
}

run_config = {
    'model_hf_id': 'meta-llama/Llama-3.2-3B-Instruct',
    'model_s3_loc': 'llama',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['BASE'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu',
    'data_location': 'data_quarterly_pit_indu.json'
}




run_name = f"{run_config['model_s3_loc']}_{run_config['dataset']}"

In [14]:
ir = modelinference.InferenceRun(run_name, run_config)

In [15]:
notebook_launcher(ir.run_multi_gpu, num_processes=torch.cuda.device_count())

Launching training on 4 GPUs.
llama
llama
llama
llama


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

Memory footprint: 6.4 GB
Waiting...
starting backtest...starting backtest...

starting backtest...
starting backtest...


100%|██████████| 8/8 [00:23<00:00,  3.14s/it]

Gathered results...Gathered results...Gathered results...Gathered results...



Finished run in 0:00:23.944181
Called Save run
called log
Saved bclarke16/tmp/fs/logs/Results_llama_data_quarterly_pit_indu.json
Run Completed!


100%|██████████| 8/8 [00:24<00:00,  3.02s/it]
[2025-02-23 13:21:35,968] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 8117 via signal SIGTERM
[2025-02-23 13:21:35,969] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 8118 via signal SIGTERM
[2025-02-23 13:21:35,970] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 8119 via signal SIGTERM


In [8]:
ir = modelinference.InferenceRun(run_name, run_config)

In [13]:
p1 = ir.create_all_prompts(True)

Requesting all datasets...
Saving data...


In [11]:
len(p1)

896

In [11]:
model = ir.load_model_from_storage(ir.model_s3_loc)

llama


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
from transformers import AutoTokenizer

In [24]:
tokenizer = AutoTokenizer.from_pretrained(ir.model_hf_id)

In [28]:
output_result = ir.run_model(p1[0]['prompt'],tokenizer,model)

running model...


In [18]:
output_result = "Based on the provided income statement and balance sheet data, I have computed the following financial ratios:\n\n1. Current Ratio: \n   Current Ratio = Current Assets / Current Liabilities\n   = 9.222000e+09 / 9.222000e+09 = 1\n\n2. Debt-to-Equity Ratio: \n   Debt-to-Equity Ratio = Total Liabilities / Total Equity\n   = 3.453300e+10 / 1.012600e+10 = 3.39\n\n3. Return on Equity (ROE): \n   ROE = Net Income / Total Equity\n   = 1.012600e+10 / 1.076400e+10 = 0.943\n\n4. Return on Assets (ROA): \n   ROA = Net Income / Total Assets\n   = 1.012600e+10 / 4.465900e+10 = 0.228\n\n5. Interest Coverage Ratio: \n   Interest Coverage Ratio = Earnings Before Interest and Taxes (EBIT) / Interest Expense\n   EBIT = Net Income + Depreciation + Amortization\n   = 1.012600e+10 + 1.679100e+10 + 1.008300e+10 = 3.700100e+10\n   Interest Expense = 1.240000e+08 + 1.110000e+08 + 1.800000e+07 = 1.960000e+08\n   Interest Coverage Ratio = 3.700100e+10 / 1.960000e+08 = 1.89\n\nBased on these ratios, I am inclined to recommend a BUY decision with a confidence score of 70%. The company has a relatively low debt-to-equity ratio and a high return on equity, indicating a strong financial position. The interest coverage ratio is also high, suggesting that the company can meet its interest payments. However, the current ratio is 1, which may indicate liquidity concerns. Further analysis of the company's cash flow and debt repayment plans is necessary to confirm this decision."

In [23]:
import json

In [20]:
def format_json(llm_output):
    # remove all the broken lines
    form = llm_output.replace('\n','')
    # Find the start and end of the JSON input
    try:
        soj = form.find('```json')
        eoj = form.find('}```')

        if eoj == -1:
            eoj = len(llm_output)
        # Pull out the additional context
        additional = form[:soj]
        additional += form[eoj + 4:]
        json_obj = json.load(form[soj + 7:eoj + 1])
        json_obj['AdditionalContext'] = additional
        return json_obj
    except:
        return llm_output

In [21]:
format_json(output_result)

"Based on the provided income statement and balance sheet data, I have computed the following financial ratios:\n\n1. Current Ratio: \n   Current Ratio = Current Assets / Current Liabilities\n   = 9.222000e+09 / 9.222000e+09 = 1\n\n2. Debt-to-Equity Ratio: \n   Debt-to-Equity Ratio = Total Liabilities / Total Equity\n   = 3.453300e+10 / 1.012600e+10 = 3.39\n\n3. Return on Equity (ROE): \n   ROE = Net Income / Total Equity\n   = 1.012600e+10 / 1.076400e+10 = 0.943\n\n4. Return on Assets (ROA): \n   ROA = Net Income / Total Assets\n   = 1.012600e+10 / 4.465900e+10 = 0.228\n\n5. Interest Coverage Ratio: \n   Interest Coverage Ratio = Earnings Before Interest and Taxes (EBIT) / Interest Expense\n   EBIT = Net Income + Depreciation + Amortization\n   = 1.012600e+10 + 1.679100e+10 + 1.008300e+10 = 3.700100e+10\n   Interest Expense = 1.240000e+08 + 1.110000e+08 + 1.800000e+07 = 1.960000e+08\n   Interest Coverage Ratio = 3.700100e+10 / 1.960000e+08 = 1.89\n\nBased on these ratios, I am incline

In [14]:
ir.save_run({'test':'1234'})

Saved s3://awmgd-prod-finml-sandbox-user/bclarke16/tmp/fs/logs/results - llama - data_quarterly_pit_indu
Run Completed!


In [10]:
helper = mh.ModelHelper('tmp/fs')

In [11]:
helper.clear_folder('llama')

In [12]:
import boto3
import json
s3 = boto3.client("s3")

In [13]:
bucket = os.environ['BQUANT_SANDBOX_USER_BUCKET']
folder = f"{os.environ['BQUANT_USERNAME']}/tmp/fs/logs"

In [26]:
s3.put_object(Body=json.dumps({'test':'0123'}), Key=f'{folder}/test.json', Bucket=bucket)

{'ResponseMetadata': {'RequestId': 'R51SF022RFPHNXM1',
  'HostId': 'mKcJOCd5U5znq7g9cBryYLOrkDxLX7p4emDnNIdj0RYizPyOVUMTDZnO2PRQTVQILvDhFsVzj1l/Ua1K0pMFlqnJda2r56jmWVR9G/49vCs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'mKcJOCd5U5znq7g9cBryYLOrkDxLX7p4emDnNIdj0RYizPyOVUMTDZnO2PRQTVQILvDhFsVzj1l/Ua1K0pMFlqnJda2r56jmWVR9G/49vCs=',
   'x-amz-request-id': 'R51SF022RFPHNXM1',
   'date': 'Sun, 23 Feb 2025 12:52:55 GMT',
   'x-amz-version-id': 'N_tWNp3h_imj758yGQrJWxpIBD0xVGy7',
   'x-amz-server-side-encryption': 'aws:kms',
   'x-amz-server-side-encryption-aws-kms-key-id': 'arn:aws:kms:us-east-1:928295279990:key/e5f1e8a7-e5c9-479d-b936-a82b325aacf0',
   'etag': '"e398cb3034762e2846b9b0bb369fae92"',
   'content-length': '0',
   'server': 'envoy',
   'x-envoy-upstream-service-time': '111'},
  'RetryAttempts': 0},
 'ETag': '"e398cb3034762e2846b9b0bb369fae92"',
 'ServerSideEncryption': 'aws:kms',
 'VersionId': 'N_tWNp3h_imj758yGQrJWxpIBD0xVGy7',
 'SSEKMSKeyId': 'arn:aws:kms:us-ea

In [35]:
from utils.logger import Logger
#logger = logger.Logger('tmp/fs')
import utils.logger as l
importlib.reload(l)

<module 'utils.logger' from '/project/utils/logger.py'>

In [36]:
logger = l.Logger('tmp/fs')

In [39]:
logger.get_list_of_logs()

['bclarke16/tmp/fs/logs/results - llama - data_quarterly_pit_indu',
 'bclarke16/tmp/fs/logs/results.json',
 'bclarke16/tmp/fs/logs/results2.json',
 'bclarke16/tmp/fs/logs/test.json',
 'bclarke16/tmp/fs/logs/test2.json',
 'bclarke16/tmp/fs/logstest.json']

In [40]:
log = logger.get_log('test2.json')

In [41]:
log

{'test2': '12345'}

In [38]:
logger.log({'test2':'12345'},'test2.json')

called log
Saved bclarke16/tmp/fs/logs/test2.json
